In [1]:
#Python packages
import numpy as np
import pandas as pd
import re
from scipy import stats


#Visualization packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('train.csv', index_col=0)

In [ ]:
##FE count of assessments per user
##ratio of assessments to other types